# Account Receivables Data (Distributor data)

In [116]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [117]:
pd.options.mode.chained_assignment = None #for not showing warning message 
pd.options.display.float_format = '{:,.4f}'.format #for setting thhe float format to this notebook

In [118]:
data = pd.read_excel(r"D:\Analytics\Undercutting\South region - GT\Data\Distributor Invoices.xlsx")

In [119]:
data.shape

(104281, 17)

In [120]:
data.columns =  data.columns.str.strip()
data.columns

Index(['Account', 'DocumentNo', 'Unnamed: 2', 'Doc..Date', 'Pstng Date',
       'CCAr', 'BusA', 'Typ', 'S', 'DD', 'Amount in DC', 'Local Crcy Amt',
       'Net Due Dt', 'Clearing', 'Text', 'LCurr', 'User Name'],
      dtype='object')

In [121]:
data.dtypes

Account           float64
DocumentNo        float64
Unnamed: 2        float64
Doc..Date          object
Pstng Date         object
CCAr               object
BusA               object
Typ                object
S                 float64
DD                float64
Amount in DC      float64
Local Crcy Amt    float64
Net Due Dt         object
Clearing           object
Text               object
LCurr              object
User Name          object
dtype: object

In [122]:
data["Net Due Dt"]                   = pd.to_datetime(data["Net Due Dt"],format='%d.%m.%Y')
data["Clearing"]                     = data["Clearing"].fillna("14.04.2020")
data["Clearing"]                     = pd.to_datetime(data["Clearing"],format='%d.%m.%Y')
data.drop(data[data["Account"].isna()].index,axis =0,inplace=True)
data["Account"]                      = data["Account"].astype(int)

In [123]:
invoicedata                          = data[["Account","Net Due Dt","Clearing","Amount in DC"]]
invoicedata                          = invoicedata[invoicedata["Net Due Dt"] <= "2020-03-31"]
invoicedata["days_diff"]             = (data["Net Due Dt"] - data["Clearing"])/timedelta(days=1)
invoicedata["No Grace Period"]            = np.where(invoicedata["days_diff"] <    0, "Yes", "No")
invoicedata["Grace period - 30days"]      = np.where(invoicedata["days_diff"] <= -30, "Yes", "No")
invoicedata["Grace period - 60days"]      = np.where(invoicedata["days_diff"] <= -60, "Yes", "No")
invoicedata["Grace period - 90days"]      = np.where(invoicedata["days_diff"] <= -90, "Yes", "No")

In [124]:
invoicedata.head(5)

,Account,Net Due Dt,Clearing,Amount in DC,days_diff,No Grace Period,Grace period - 30days,Grace period - 60days,Grace period - 90days
3,2000001,2020-03-30,2020-03-30,"-697,775.2800",0.0000,No,No,No,No
4,2000001,2020-03-28,2020-03-28,"-15,167.0100",0.0000,No,No,No,No
5,2000001,2020-03-28,2020-03-28,"15,167.0100",0.0000,No,No,No,No
6,2000001,2020-03-26,2020-03-26,"20,310.0000",0.0000,No,No,No,No
7,2000001,2020-03-26,2020-03-26,"-20,310.0000",0.0000,No,No,No,No


In [125]:
count_dist                    = invoicedata.groupby(["Account"]).agg(Count = ("Account","count")).reset_index()
days_0                        = invoicedata.groupby("Account")["No Grace Period"].apply(lambda x: (x == 'Yes').mean())
days_0                        = pd.DataFrame(days_0).reset_index()
days_30                       = invoicedata.groupby("Account")["Grace period - 30days"].apply(lambda x: (x == 'Yes').mean())
days_30                       = pd.DataFrame(days_30).reset_index()
# days_30["Less than 30days"]   = days_30["Less than 30days"] * 100
days_60                       = invoicedata.groupby("Account")["Grace period - 60days"].apply(lambda x: (x == 'Yes').mean())
days_60                       = pd.DataFrame(days_60).reset_index()
# days_60["Less than 60days"]   = days_60["Less than 60days"] * 100
days_90                       = invoicedata.groupby("Account")["Grace period - 90days"].apply(lambda x: (x == 'Yes').mean())
days_90                       = pd.DataFrame(days_90).reset_index()
# days_90["Less than 90days"]   = days_90["Less than 90days"] * 100
dist_default                  = count_dist.merge(days_0,on="Account").merge(days_30,on="Account").merge(days_60,on="Account").merge(days_90,on="Account")

In [126]:
dist_default.head(2)

,Account,Count,No Grace Period,Grace period - 30days,Grace period - 60days,Grace period - 90days
0,2000001,868,0.4090,0.0046,0.0023,0.0023
1,2000002,526,0.4316,0.0190,0.0076,0.0038


In [127]:
dist_default.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\dist_default.xlsx",index = False)

In [128]:
invoicedata.to_excel(r"D:\Analytics\Undercutting\South region - GT\Outfiles\invoicedata.xlsx",index = False)

In [129]:
%store dist_default

Stored 'dist_default' (DataFrame)


In [113]:
data["Account"].nunique()

448